This is the first part of my project to build a movie recommendation system: retrieving and cleaning the relevant data.

I will be using the full MovieLens dataset comprising over 25 million movie/user rating pairs, in addition to IMDb data that I will scrape from the web using custom Python code.

First, I will load the relevant Python libraries:

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup

Now, I will load the relevant datasets and examine them:

In [3]:
ratings = pd.read_csv('ratings.csv')
links = pd.read_csv('links.csv')

print(ratings.head())
print(links.head())

   userId  movieId  rating   timestamp
0       1      307     3.5  1256677221
1       1      481     3.5  1256677456
2       1     1091     1.5  1256677471
3       1     1257     4.5  1256677460
4       1     1449     4.5  1256677264
   movieId  imdbId   tmdbId
0        1  114709    862.0
1        2  113497   8844.0
2        3  113228  15602.0
3        4  114885  31357.0
4        5  113041  11862.0


Timestamp data and tmdbId will not be necessary for this project, so I will remove them:

In [5]:
ratings = ratings.iloc[:,:-1]
links = links.iloc[:,:-1]

Next, I will check the data quality of the files:

In [6]:
print(ratings.info())
print(links.info())

print(ratings.isnull().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27753444 entries, 0 to 27753443
Data columns (total 3 columns):
 #   Column   Dtype  
---  ------   -----  
 0   userId   int64  
 1   movieId  int64  
 2   rating   float64
dtypes: float64(1), int64(2)
memory usage: 635.2 MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 58098 entries, 0 to 58097
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   movieId  58098 non-null  int64
 1   imdbId   58098 non-null  int64
dtypes: int64(2)
memory usage: 907.9 KB
None
userId     0
movieId    0
rating     0
dtype: int64


There appear to be no null or incorrect values in either dataset, as expected.

However, the ratings set is very large, too large for many of the analyses I want to perform, so I will restrict the dataset to users who have watched over 1000 movies, and movies which have been watched by at least 50 users. 

In [7]:
print(ratings.shape)

bool_filter_users = []
value_counts_users = ratings['userId'].value_counts()

for user in ratings['userId']:
    if value_counts_users[user] >= 1000:
        bool_filter_users.append(True)
    else:
        bool_filter_users.append(False)

ratings = ratings[bool_filter_users]

print(ratings.shape)

bool_filter_movies = []
value_counts_movies = ratings['movieId'].value_counts()

for movie in ratings['movieId']:
    if value_counts_movies[movie] >= 50:
        bool_filter_movies.append(True)
    else:
        bool_filter_movies.append(False)

ratings = ratings[bool_filter_movies]

print(ratings.shape)

(27753444, 3)
(4295475, 3)
(4002692, 3)


Now I will save the new files.

In [8]:
ratings.to_csv('ratings1000.csv')
links = links[links['movieId'].isin(ratings['movieId'].unique())]
print(links.shape)
links.to_csv('links_reduced.csv')

(10206, 2)


The 'imdbId' column in links does not match the format of the IDs used on the IMDb website, so I will use the following function and code to fix this.

In [8]:
def convert(x):
    x = str(x)
    while len(x) < 7:
        x = '0' + x
    x = 'tt' + x
    return x

In [9]:
links['imdbId_converted'] = links['imdbId'].apply(lambda x: convert(x))
links = links.drop('imdbId',axis=1)
links.to_csv('links_reduced.csv')

Now it's time to scrape IMDb! The following code uses the requests package and BeautifulSoup to acquire the top 3 most 'helpful' reviews for each movie in the dataset. I needed to examine the IMDb website beforehand to determine how to construct this code.

WARNING: This code takes a very long time to run.

In [14]:
reviews_url_1 = "https://www.imdb.com/title/"
reviews_url_2 = "/reviews?ref_=tt_ql_op_3"

imdb = links[['movieId','imdbId_converted']]
df = pd.DataFrame(columns=['movieId','imdbId','review_text'])

for index,row in imdb.iterrows():
    movieid = imdb.iloc[index,0]
    idval = imdb.iloc[index,1]
    address = reviews_url_1 + idval + reviews_url_2
    print(address)
    page = requests.get(address).text
    bsObj = BeautifulSoup(page,features='lxml')
    if bsObj.find('div') == None:
        continue
    reviews = bsObj.find_all('div',class_='text')
    review_text = ""
    for i in range(min(len(reviews),3)):
        review_text = review_text + reviews[i].get_text()
    
    
    
    df = df.append({'movieId':movieid,'imdbId': idval,'review_text':review_text},ignore_index=True)
df.to_csv('movie_reviews.csv')

https://www.imdb.com/title/tt0114709/reviews?ref_=tt_ql_op_3
https://www.imdb.com/title/tt0113497/reviews?ref_=tt_ql_op_3
https://www.imdb.com/title/tt0113228/reviews?ref_=tt_ql_op_3


KeyboardInterrupt: 

Now I scrape IMDb again to acquire all other relevant metadata about eac=h movie. Of particular interest are the titles, imdb scores, number of votes, the genres, and the ratings.

Again, this code takes a very long time to run.

In [15]:
mainurl = "https://www.imdb.com/title/"

#imdb = pd.read_csv("links_reduced.csv")

imdb = links['imdbId_converted']

#ids_titles = dict()
rating_cats = ['Passed','G','PG','PG-13','R','NC-17']
df = pd.DataFrame(columns=['imdbId','title','age_rating','time_minutes','genres','imdb_score','imdb_votes','director','actors','summary','language'])
#df = pd.read_csv('id_title6.csv')
#df = df.iloc[:,-11:]
for idval in imdb.to_list():
    page = requests.get(mainurl + idval).text
    bsObj = BeautifulSoup(page,features='lxml')
    if bsObj.find('h1') == None:
        continue
    title = bsObj.find('h1').get_text()
    print(title)
    info = bsObj.find('div',class_='subtext')
    if info == None:
        continue
    info = info.get_text().split('|')
    if len(info) != 4:
        continue
    rating = info[0]
    rating = rating.replace(" ", "")
    rating = rating.replace("\n","")
    time = info[1]
    time = time.replace("\n","")
    time = time.replace(" ", "")
    time = time.split('h')
    
    score = bsObj.find('span', itemprop='ratingValue').get_text()
    votes = bsObj.find('span', itemprop='ratingCount').get_text()
    
    stars = bsObj.find_all('div',class_='credit_summary_item')
    
    director = 'None'
    actors = 'None'
    
    if len(stars) == 3:
        director = stars[0].find_all('a')[0].get_text()
        director = director.rstrip()
        director = director.replace('\n','')
        actors = stars[2].get_text()
        actors = actors.replace('Stars:','')
        actors = actors.split('|')
        actors = actors[0].lstrip()
    summary = bsObj.find('div',class_='summary_text').get_text().replace('\n','').lstrip()
    print(summary)
        
    
    if len(time) == 1:
        if 'min' in time[0]:
            time_minutes = int(time[0].split('min')[0])
        else:
            time_minutes = int(time[0])*60
    else:
        if 'min' in time[1]:
            time_minutes = int(time[0])*60 + int(time[1].split('min')[0])
        else:
            time_minutes = int(time[0])*60

    if rating not in rating_cats:
        rating = 'Unknown'
    
    txtblock = bsObj.find_all('div',class_='txt-block')
    seemore = bsObj.find_all('div',class_='see-more')
    
    language = 'Unknown'
    genres_list = ""
    
    for t in txtblock:
        h4 = t.find('h4')
        if h4 != None:
            h4text = h4.get_text()
        if h4text == 'Language:':
            language = t.find_all('a')[0].get_text()
        
    
    for t in seemore:
        h4 = t.find('h4')
        if h4 !=None:
            h4text = h4.get_text()
        if h4text == 'Genres:':
            genres = t.find_all('a')
            for genre in genres:
                genretext = genre.get_text().replace(' ','')
                genres_list = genres_list + genretext + ','
    genres_list = genres_list[:-1]
        
    
    
    df = df.append({'imdbId': idval,'title':title, 'age_rating':rating, 'time_minutes':time_minutes, 'genres':genres_list, 'imdb_score':score, 'imdb_votes':votes, 'director': director, 'actors':actors, 'summary':summary, 'language':language},ignore_index=True)
    print(df)
df.to_csv('id_title.csv')

Toy Story (1995) 
A cowboy doll is profoundly threatened and jealous when a new spaceman figure supplants him as top toy in a boy's room.            
      imdbId              title age_rating time_minutes  \
0  tt0114709  Toy Story (1995)           G           81   

                                      genres imdb_score imdb_votes  \
0  Animation,Adventure,Comedy,Family,Fantasy        8.3    877,792   

        director                              actors  \
0  John Lasseter  Tom Hanks, Tim Allen, Don Rickles    

                                             summary language  
0  A cowboy doll is profoundly threatened and jea...  English  
Jumanji (1995) 
When two kids find and play a magical board game, they release a man trapped in it for decades - and a host of dangers that can only be stopped by finishing the game.            
      imdbId              title age_rating time_minutes  \
0  tt0114709  Toy Story (1995)           G           81   
1  tt0113497    Jumanji (1995)      

Sabrina (1995) 
An ugly duckling having undergone a remarkable change, still harbors feelings for her crush: a carefree playboy, but not before his business-focused brother has something to say about it.            
      imdbId                                title age_rating time_minutes  \
0  tt0114709                    Toy Story (1995)           G           81   
1  tt0113497                      Jumanji (1995)          PG          104   
2  tt0113228             Grumpier Old Men (1995)       PG-13          101   
3  tt0114885            Waiting to Exhale (1995)           R          124   
4  tt0113041  Father of the Bride Part II (1995)          PG          106   
5  tt0113277                         Heat (1995)           R          170   
6  tt0114319                      Sabrina (1995)          PG          127   

                                      genres imdb_score imdb_votes  \
0  Animation,Adventure,Comedy,Family,Fantasy        8.3    877,792   
1            Adventure,Come

GoldenEye (1995) 
Years after a friend and fellow 00 agent is killed on a joint mission, a secret space based weapons program known as "GoldenEye" is stolen. James Bond sets out to stop a Russian crime syndicate from using the weapon.            
      imdbId                                title age_rating time_minutes  \
0  tt0114709                    Toy Story (1995)           G           81   
1  tt0113497                      Jumanji (1995)          PG          104   
2  tt0113228             Grumpier Old Men (1995)       PG-13          101   
3  tt0114885            Waiting to Exhale (1995)           R          124   
4  tt0113041  Father of the Bride Part II (1995)          PG          106   
5  tt0113277                         Heat (1995)           R          170   
6  tt0114319                      Sabrina (1995)          PG          127   
7  tt0112302                 Tom and Huck (1995)          PG           97   
8  tt0114576                 Sudden Death (1995)           R

Balto (1995) 
An outcast Husky risks his life with other sled dogs to prevent a deadly epidemic from ravaging Nome, Alaska.            
       imdbId                                title age_rating time_minutes  \
0   tt0114709                    Toy Story (1995)           G           81   
1   tt0113497                      Jumanji (1995)          PG          104   
2   tt0113228             Grumpier Old Men (1995)       PG-13          101   
3   tt0114885            Waiting to Exhale (1995)           R          124   
4   tt0113041  Father of the Bride Part II (1995)          PG          106   
5   tt0113277                         Heat (1995)           R          170   
6   tt0114319                      Sabrina (1995)          PG          127   
7   tt0112302                 Tom and Huck (1995)          PG           97   
8   tt0114576                 Sudden Death (1995)           R          111   
9   tt0113189                    GoldenEye (1995)       PG-13          130   
10  tt

Cutthroat Island (1995) 
A female pirate and her companion race against their rivals to find a hidden island that contains a fabulous treasure.            
       imdbId                                title age_rating time_minutes  \
0   tt0114709                    Toy Story (1995)           G           81   
1   tt0113497                      Jumanji (1995)          PG          104   
2   tt0113228             Grumpier Old Men (1995)       PG-13          101   
3   tt0114885            Waiting to Exhale (1995)           R          124   
4   tt0113041  Father of the Bride Part II (1995)          PG          106   
5   tt0113277                         Heat (1995)           R          170   
6   tt0114319                      Sabrina (1995)          PG          127   
7   tt0112302                 Tom and Huck (1995)          PG           97   
8   tt0114576                 Sudden Death (1995)           R          111   
9   tt0113189                    GoldenEye (1995)       PG-13   

...

Ace Ventura: When Nature Calls (1995) 
Ace Ventura, Pet Detective, returns from a spiritual quest to investigate the disappearance of a rare white bat, the sacred animal of a tribe in Africa.            
       imdbId                                   title age_rating time_minutes  \
0   tt0114709                       Toy Story (1995)           G           81   
1   tt0113497                         Jumanji (1995)          PG          104   
2   tt0113228                Grumpier Old Men (1995)       PG-13          101   
3   tt0114885               Waiting to Exhale (1995)           R          124   
4   tt0113041     Father of the Bride Part II (1995)          PG          106   
5   tt0113277                            Heat (1995)           R          170   
6   tt0114319                         Sabrina (1995)          PG          127   
7   tt0112302                    Tom and Huck (1995)          PG           97   
8   tt0114576                    Sudden Death (1995)           R   

Get Shorty (1995) 
A mobster travels to Hollywood to collect a debt, and discovers that the movie business is much the same as his current job.            
       imdbId                                   title age_rating time_minutes  \
0   tt0114709                       Toy Story (1995)           G           81   
1   tt0113497                         Jumanji (1995)          PG          104   
2   tt0113228                Grumpier Old Men (1995)       PG-13          101   
3   tt0114885               Waiting to Exhale (1995)           R          124   
4   tt0113041     Father of the Bride Part II (1995)          PG          106   
5   tt0113277                            Heat (1995)           R          170   
6   tt0114319                         Sabrina (1995)          PG          127   
7   tt0112302                    Tom and Huck (1995)          PG           97   
8   tt0114576                    Sudden Death (1995)           R          111   
9   tt0113189                     

Assassins (1995) 
Professional hit-man Robert Rath wants to fulfill a few more contracts before retiring but unscrupulous ambitious newcomer hit-man Miguel Bain keeps killing Rath's targets.            
       imdbId                                   title age_rating time_minutes  \
0   tt0114709                       Toy Story (1995)           G           81   
1   tt0113497                         Jumanji (1995)          PG          104   
2   tt0113228                Grumpier Old Men (1995)       PG-13          101   
3   tt0114885               Waiting to Exhale (1995)           R          124   
4   tt0113041     Father of the Bride Part II (1995)          PG          106   
5   tt0113277                            Heat (1995)           R          170   
6   tt0114319                         Sabrina (1995)          PG          127   
7   tt0112302                    Tom and Huck (1995)          PG           97   
8   tt0114576                    Sudden Death (1995)           R    

Leaving Las Vegas (1995) 
Ben Sanderson, a Hollywood screenwriter who lost everything because of his alcoholism, arrives in Las Vegas to drink himself to death. There, he meets and forms an uneasy friendship and non-interference pact with prostitute Sera.            
       imdbId                                   title age_rating time_minutes  \
0   tt0114709                       Toy Story (1995)           G           81   
1   tt0113497                         Jumanji (1995)          PG          104   
2   tt0113228                Grumpier Old Men (1995)       PG-13          101   
3   tt0114885               Waiting to Exhale (1995)           R          124   
4   tt0113041     Father of the Bride Part II (1995)          PG          106   
5   tt0113277                            Heat (1995)           R          170   
6   tt0114319                         Sabrina (1995)          PG          127   
7   tt0112302                    Tom and Huck (1995)          PG           97   
8  

KeyboardInterrupt: 

Some minor massaging of the acquired data:

In [4]:
df = pd.read_csv('id_title.csv')
links = pd.read_csv('links_reduced.csv')

print(links.head())

links = links.iloc[:,2:]

df = df.merge(links,how='inner',left_on='imdbId',right_on='imdbId_converted')
df = df.drop(['imdbId_converted'],axis=1)
df = df.iloc[:,1:]

print(df.columns)
print(df.shape)

   Unnamed: 0  Unnamed: 0.1  movieId imdbId_converted
0           0             0        1        tt0114709
1           1             1        2        tt0113497
2           2             2        3        tt0113228
3           3             3        4        tt0114885
4           4             4        5        tt0113041
Index(['imdbId', 'title', 'age_rating', 'time_minutes', 'genres', 'imdb_score',
       'imdb_votes', 'director', 'actors', 'summary', 'language', 'movieId'],
      dtype='object')
(9987, 12)


I will call df.info to see some basic info about my new data.

In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9987 entries, 0 to 9986
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   imdbId        9987 non-null   object 
 1   title         9987 non-null   object 
 2   age_rating    9987 non-null   object 
 3   time_minutes  9987 non-null   int64  
 4   genres        9987 non-null   object 
 5   imdb_score    9987 non-null   float64
 6   imdb_votes    9987 non-null   object 
 7   director      9987 non-null   object 
 8   actors        9987 non-null   object 
 9   summary       9987 non-null   object 
 10  language      9987 non-null   object 
 11  movieId       9987 non-null   int64  
dtypes: float64(1), int64(2), object(9)
memory usage: 1014.3+ KB
None


No nulls, good. However, the web-scraping code was pretty complex and has a lot of outputs, so let's invesitgate further and see if there are any duplicates.

In [8]:
for column in df.columns:
    print(column)
    print(len(df[df[column].duplicated()]))

imdbId
0
title
4
age_rating
9980
time_minutes
9750
genres
8438
imdb_score
9909
imdb_votes
710
director
6484
actors
380
summary
2
language
9925
movieId
0


Interesting. There are four movies with duplicated titles (including date). This is certainly possible, however there are also two sets of movies with identical text summaries. This seems less likely.

In [9]:
print(df[df['title'].duplicated(keep=False)])
print(df[df['summary'].duplicated(keep=False)])

         imdbId                                    title age_rating  \
4495  tt0290538  Confessions of a Dangerous Mind (2002)           R   
5394  tt0108906                       Riget                 Unknown   
6204  tt0127392                       Riget                 Unknown   
6270  tt0355987                 Salem's Lot                 Unknown   
6381  tt0079844                 Salem's Lot                      PG   
6544  tt0407304                War of the Worlds (2005)       PG-13   
7663  tt0449040                War of the Worlds (2005)           R   
9610  tt0270288  Confessions of a Dangerous Mind (2002)           R   

      time_minutes                                         genres  imdb_score  \
4495           113  Biography,Comedy,Crime,Drama,Romance,Thriller         7.0   
5394           561            Comedy,Drama,Fantasy,Horror,Mystery         8.3   
6204           561            Comedy,Drama,Fantasy,Horror,Mystery         8.3   
6270           181                  

Based on these results, it seems as though two of the cases are legitimately different movies with the same title from the same year, but that two of the cases are indeed duplicates, Riget and Confessions of a Dangerous Mind. I will drop the duplicates of both of these.

In [10]:
df = df.drop_duplicates('summary',ignore_index=True)
print(df.shape)

(9985, 12)


Now it's time to examine the data on reviews I just collected:

In [13]:
reviews = pd.read_csv('movie_reviews.csv')
print(reviews.head())
print(reviews.info())

   Unnamed: 0  movieId     imdbId  \
0           0        1  tt0114709   
1           1        2  tt0113497   
2           2        3  tt0113228   
3           3        4  tt0114885   
4           4        5  tt0113041   

                                         review_text  
0  Andy's toys live a reasonable life of fun and ...  
1  I have not seen this movie in more then a deca...  
2  Some people see this as an inferior sequel to ...  
3  I read some of the other comments comcerning t...  
4  I enjoyed this film, as I did Father of the Br...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10115 entries, 0 to 10114
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Unnamed: 0   10115 non-null  int64 
 1   movieId      10115 non-null  int64 
 2   imdbId       10115 non-null  object
 3   review_text  10114 non-null  object
dtypes: int64(2), object(2)
memory usage: 316.2+ KB
None


There seems to be a single entry for which there were no reviews. Since reviews are essential to this project, I will drop this row.

I will also establish which movies are contained in both datasets, and make sure both have 100% overlap by removing inconsistent entries.

In [14]:
print(reviews[reviews['review_text'].isna()])

     Unnamed: 0  movieId     imdbId review_text
579         579      720  tt0118114         NaN


In [24]:
reviews = reviews.dropna()
print(reviews.shape)

combined = reviews[['movieId']].merge(df[['movieId']],how='inner',on='movieId')
print(combined.shape)

(10114, 4)
(9985, 1)


In [25]:
reviews = reviews[reviews['movieId'].isin(combined['movieId'])]
print(reviews.shape)

(9985, 4)


I believe that I have addressed the necessary issues for this part of the process. Time to save my files:

In [26]:
df.to_csv('id_title_clean.csv')
reviews.to_csv('movie_reviews_clean.csv')

The next section will involve graphical and statistical exploration of the data, in addition to drawing insights from clustering.